In [31]:

## Import libraries section ##

import numpy as np
import cv2
import os
import math
from LFR_utils import read_poses_and_images,pose_to_virtualcamera, init_aos, init_window
import LFR_utils as utils
import pyaos
import glm
import glob
import re


## path to where the results will be stored 

user_name = os.getlogin()

base_path = 'C:\\Users\\{}\\Documents\\computervision\\'.format(user_name)

print(f"Starting with base path: {base_path}")

Download_Location = base_path    ## Enter path to the directory where you want to save the results.

Integral_Path = os.path.join(Download_Location,'integrals') # Note that your results will be saved to this integrals folder.

print(f"Results will be saved to: {Integral_Path}")

# Check if the directory already exists
if not os.path.exists(Integral_Path):
    os.mkdir(Integral_Path)
else:
    print(f"The directory '{Integral_Path}' already exists.")

set_folder = base_path+r'\\ImageRestoration-CV\\LFR\\python'
print(f"Path to your LFR/python directory - it must be here: {set_folder}")


Starting with base path: C:\Users\marti\Documents\computervision\
C:\Users\marti\Documents\computervision\
The directory 'C:\Users\marti\Documents\computervision\integrals' already exists.


In [32]:
#############################Start the AOS Renderer###############################################################
w,h,fovDegrees = 512, 512, 50 # # resolution and field of view. This should not be changed.
render_fov = 50

if 'window' not in locals() or window == None:                                    
    window = pyaos.PyGlfwWindow( w, h, 'AOS' )  
     
aos = pyaos.PyAOS(w,h,fovDegrees) 


aos.loadDEM( os.path.join(set_folder,'zero_plane.obj'))

####################################################################################################################

In [33]:
#############################Create Poses for Initial Positions###############################################################

# Below are certain functions required to convert the poses to a certain format to be compatabile with the AOS Renderer.

def eul2rotm(theta) :
    s_1 = math.sin(theta[0])
    c_1 = math.cos(theta[0]) 
    s_2 = math.sin(theta[1]) 
    c_2 = math.cos(theta[1]) 
    s_3 = math.sin(theta[2]) 
    c_3 = math.cos(theta[2])
    rotm = np.identity(3)
    rotm[0,0] =  c_1*c_2
    rotm[0,1] =  c_1*s_2*s_3 - s_1*c_3
    rotm[0,2] =  c_1*s_2*c_3 + s_1*s_3

    rotm[1,0] =  s_1*c_2
    rotm[1,1] =  s_1*s_2*s_3 + c_1*c_3
    rotm[1,2] =  s_1*s_2*c_3 - c_1*s_3

    rotm[2,0] = -s_2
    rotm[2,1] =  c_2*s_3
    rotm[2,2] =  c_2*c_3        

    return rotm

def createviewmateuler(eulerang, camLocation):
    
    rotationmat = eul2rotm(eulerang)
    translVec =  np.reshape((-camLocation @ rotationmat),(3,1))
    conjoinedmat = (np.append(np.transpose(rotationmat), translVec, axis=1))
    return conjoinedmat

def divide_by_alpha(rimg2):
        a = np.stack((rimg2[:,:,3],rimg2[:,:,3],rimg2[:,:,3]),axis=-1)
        return rimg2[:,:,:3]/a

def pose_to_virtualcamera(vpose ):
    vp = glm.mat4(*np.array(vpose).transpose().flatten())
    #vp = vpose.copy()
    ivp = glm.inverse(glm.transpose(vp))
    #ivp = glm.inverse(vpose)
    Posvec = glm.vec3(ivp[3])
    Upvec = glm.vec3(ivp[1])
    FrontVec = glm.vec3(ivp[2])
    lookAt = glm.lookAt(Posvec, Posvec + FrontVec, Upvec)
    cameraviewarr = np.asarray(lookAt)
    #print(cameraviewarr)
    return cameraviewarr  

In [97]:
from tqdm import tqdm

import shutil

from image_group import ImageGroup

def copy_and_rename_file(src, new_name):
    assert os.path.exists(src), f"Could not find file {src}"
    os.makedirs(os.path.dirname(new_name), exist_ok=True)
    shutil.copy(src, new_name)

########################## Below we generate the poses for rendering #####################################
# This is based on how renderer is implemented. 

def load_filenames(base_path):
    all_files = os.listdir(base_path)
    img_group_dict = {}

    # Regex to extract image_index and pose_index
    file_pattern = re.compile(r"([0123456789])_(\d+)_pose_(\d+)_thermal\.png")

    for file in all_files:
        match = file_pattern.match(file)
        if match:
            image_prefix = match.group(1)
            image_index = match.group(2)
            full_image_index = ImageGroup.calc_full_image_index(image_prefix, image_index)
            if full_image_index not in img_group_dict:
                img_group_dict[full_image_index] = ImageGroup(full_image_index)
            img_group_dict[full_image_index].add_filename(os.path.join(base_path, file))
    return img_group_dict

def create_output_directory(base_output_path, part_name):
    output_dir = os.path.join(base_output_path, part_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

def create_integral_image(file_list, output_image_name, Focal_plane, progress_bar):

    assert len(file_list) == 11, f"Expected 11 images but got {len(file_list)}"
    Numberofimages = 11
    # ref_loc is the reference location or the poses of the images. The poses are the same for the dataset and therefore only the images have to be replaced.
    ref_loc = [[5,4,3,2,1,0,-1,-2,-3,-4,-5],[0,0,0,0,0,0,0,0,0,0,0]]   # These are the x and y positions of the images. It is of the form [[x_positions],[y_positions]]

    altitude_list = [35,35,35,35,35,35,35,35,35,35,35] # [Z values which is the height]

    center_index = 5  # this is important, this will be the pose index at which the integration should happen. For example if you have 5 images, lets say you want to integrate all 5 images to the second image position. Then your center_index is 1 as index starts from zero.

    site_poses = []
    for i in range(Numberofimages):
        EastCentered = (ref_loc[0][i] - 0.0) #Get MeanEast and Set MeanEast
        NorthCentered = (0.0 - ref_loc[1][i]) #Get MeanNorth and Set MeanNorth
        M = createviewmateuler(np.array([0.0, 0.0, 0.0]),np.array( [ref_loc[0][i], ref_loc[1][i], - altitude_list[i]] ))
        #print('m',M)
        ViewMatrix = np.vstack((M, np.array([0.0,0.0,0.0,1.0],dtype=np.float32)))
        #print(ViewMatrix)
        camerapose = np.asarray(ViewMatrix.transpose(),dtype=np.float32)
        #print(camerapose)
        site_poses.append(camerapose)  # site_poses is a list now containing all the poses of all the images in a certain format that is accecpted by the renderer.
        
    imagelist = []

    for file in file_list:
        img = cv2.imread(file)
        assert img is not None, f"Could not read image {file}"
        imagelist.append(img)
        
    #############################Read the generated images from the simulator and store in a list ###############################################################
        

    aos.clearViews()   # Every time you call the renderer you should use this line to clear the previous views  
    for i in range(len(imagelist)):
            aos.addView(imagelist[i], site_poses[i], "DEM BlobTrack")  # Here we are adding images to the renderer one by one.
    aos.setDEMTransform([0, 0, Focal_plane*-1]) #This is the focal plane. You can change this to any value between -5 to 5, above ground is negative

    proj_RGBimg = aos.render(pose_to_virtualcamera(site_poses[center_index]), render_fov)
    tmp_RGB = divide_by_alpha(proj_RGBimg)
    progress_bar.set_description("Writing image to {}".format(output_image_name))
    cv2.imwrite(output_image_name, tmp_RGB)   # Final result. Check the integral result in the integrals folder.


base_paths = ["Part2 3"] # already processed for now - "Part1", "Part1 2", "Part1 3", "Part2", "Part2 2", 
base_output_path = Integral_Path

for part_name in base_paths:
    print(f"Processing part : {part_name}")
    part_base_path = os.path.join(base_path, part_name)
    output_path = create_output_directory(base_output_path, part_name)

    img_group_dict = load_filenames(part_base_path)

    image_idx_list = list(img_group_dict.keys())

    image_idx_list = sorted(image_idx_list, key=lambda x: x)

    invalid_img_groups = []
    invalid_img_group_reasons = {}

    for image_idx in image_idx_list:
        img_group : ImageGroup = img_group_dict[image_idx]
        img_group.initialize_and_validate(output_path)
        if not img_group.valid:
            invalid_img_groups.append(img_group)
            if img_group.invalid_reason not in invalid_img_group_reasons:
                invalid_img_group_reasons[img_group.invalid_reason] = {
                    "count": 0,
                    "first_10": []
                }
            invalid_img_group_reasons[img_group.invalid_reason]["count"] += 1
            if len(invalid_img_group_reasons[img_group.invalid_reason]["first_10"]) < 10:
                invalid_img_group_reasons[img_group.invalid_reason]["first_10"].append(img_group)


    print(f"Found {len(invalid_img_groups)} invalid image groups which will not be processed for {part_name}, image indices: {[x.image_index for x in invalid_img_groups]}")
    for reason, itm in invalid_img_group_reasons.items():
        print(f"Invalidity reason: {reason} - Count: {itm['count']} - first 10 instances: {[x.image_index for x in itm['first_10']]} {[x.original_ground_truth_file for x in itm['first_10']]} {[x.original_parameter_file for x in itm['first_10']]}")

    image_idx_list_with_progress_bar = tqdm(image_idx_list, desc=f"Processing images in {part_name}")

    for image_idx in image_idx_list_with_progress_bar:
        img_group : ImageGroup = img_group_dict[image_idx]

        if not img_group.valid:
            continue

        copy_and_rename_file(img_group.original_parameter_file, img_group.new_parameter_file)
        copy_and_rename_file(img_group.original_ground_truth_file, img_group.new_ground_truth_file)

        for focal_stack_img_index in range(0, 31) :
            create_integral_image(img_group.filenames, img_group.output_image_name(focal_stack_img_index), focal_stack_img_index * 0.1, image_idx_list_with_progress_bar)
        

Processing part : Part2 3
Found 87 invalid image groups which will not be processed for Part2 3, image indices: [205576, 205604, 205628, 205645, 205680, 205744, 205888, 206123, 206128, 206137, 206226, 206296, 206300, 206331, 206336, 206338, 206361, 206364, 206371, 206401, 206409, 206426, 206431, 206564, 206607, 206913, 206976, 207124, 207206, 207257, 207339, 207400, 207417, 207433, 207458, 207464, 207503, 207549, 207608, 207661, 207715, 207759, 207779, 207808, 207865, 207898, 207900, 207918, 207955, 208011, 208065, 208157, 208380, 208469, 208577, 208588, 208820, 208836, 208838, 208845, 208885, 209131, 209227, 209238, 209508, 209522, 209537, 209580, 209585, 209697, 209755, 209897, 209900, 210015, 210059, 210060, 210370, 210453, 210475, 210493, 210539, 210614, 210860, 210883, 210887, 210909, 210928]
Invalidity reason: Could not find ground truth file.  - Count: 87 - first 10 instances: [205576, 205604, 205628, 205645, 205680, 205744, 205888, 206123, 206128, 206137] ['C:\\Users\\marti\\Do

Writing image to C:\Users\marti\Documents\computervision\integrals\Part2 3\207970\207970_05.png:  44%|████▍     | 2358/5334 [1:39:37<2:17:59,  2.78s/it]

In [ ]:
t